In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("parquet")\
        .load("abfss://bronze@adlsolympics.dfs.core.windows.net/athletes")

In [0]:
display(df)

In [0]:
df = df.fillna({'birth_place': 'unknown', 'birth_country': 'India'})

In [0]:
df_filter = df.filter((col('current') == True) & col('name').isin('KUMEL SAADI','MPALI Adam','ALI JASIM','LO Ryan')).display()

In [0]:
df = df.withColumn('height',col('height').cast(FloatType()))\
        .withColumn('weight',col('weight').cast(FloatType()))

In [0]:
df.display()

In [0]:
df_sorted = df.sort('height','weight',ascending = [0,1]).filter(col('weight')>0)
df_sorted.display()

In [0]:

df = df.withColumn('nationality', regexp_replace('nationality','United States','US'))
df.display()

In [0]:
df_agg = df.groupBy('code').agg(count('*').alias('total_count')).filter(col('total_count') > 1)

df_agg.display()

In [0]:
df = df.withColumnRenamed('name','fullname')
display(df)

In [0]:
df = df.withColumn('occupation', split(col('occupation').cast('string'), ','))
display(df)

In [0]:
df.columns

In [0]:
df_final = df.select('code',
 'current',
 'fullname',
 'name_short',
 'name_tv',
 'gender',
 'function',
 'country_code',
 'country',
 'country_long',
 'nationality',
 'nationality_long',
 'nationality_code',
 'height',
 'weight')
df_final.display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn('cumulative_sum', sum('height').over(Window.partitionBy('country').orderBy('height').rowsBetween(Window.unboundedPreceding, Window.currentRow)))
df.display()

In [0]:
df_final_filter = df.select('country','height','cumulative_sum').filter(col('height') > 0).display()

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path", "abfss://silver@adlsolympics.dfs.core.windows.net/athletes")\
    .saveAsTable("olympics_cata.silver.athletes")